In [2]:
import spacy
import timeit
import math
import pandas as pd
import matplotlib.pyplot as plt
from os import path
from collections import Counter
from lxml import etree
from glob import glob
from unicodedata import normalize

/home/humanum/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
%matplotlib inline

In [3]:
nlp = spacy.load('fr_core_news_lg')

In [4]:
nlp.disable_pipes('ner')

['ner']

In [25]:
nlp.max_length = 4000000

In [49]:
path_name = 'corpus_main_txt/*.txt'
#path_name = 'corpus_temp/*.txt'
window = 1000
nombre_bigrammes = 100

In [7]:
def clean_text(txt):
    txt_res = normalize("NFKD", txt.replace('\xa0', ' '))
    txt_res = txt_res.replace('\\xa0', '')
    return txt_res

In [8]:
def pipeline_spacy(path):
    pos_ko = ["NUM", "X", "SYM", "PUNCT", "SPACE"]
    list_lemma = []
    list_pos = []
    nombre_tokens = 0
    with open(path, encoding="utf8") as file:
        text = file.readlines()
        text_clean = clean_text(str(text).lower())
        docs = nlp(text_clean)
        nombre_tokens += len(docs)
        for token in docs:
            #si le token est bien un mot on récupère son lemme
            if token.pos_ not in pos_ko:
                list_lemma.append(token.lemma_)
                list_pos.append(token.pos_)

    return list_lemma, list_pos, nombre_tokens

In [9]:
def bigrammize(list_token):
    """fonction qui prend en parametre une liste de tokens et retourne une liste de bi-grammes"""
    list_bigram = []
    for indice_token in range(len(list_token)-1):
        bigram = list_token[indice_token]+'_'+list_token[indice_token+1]
        list_bigram.append(bigram)
    return list_bigram

In [10]:
def trigrammize(list_token):
    """fonction qui prend en parametre une liste de tokens et retourne une liste de tri-grammes"""
    list_trigram = []
    for indice_token in range(len(list_token)-2):
        trigram = list_token[indice_token]+'_'+list_token[indice_token+1]+'_'+list_token[indice_token+2]
        list_trigram.append(trigram)
    return list_trigram

In [11]:
def rollingntokens(list_tokens, n):
    """fonction qui prend en parametre une liste de tokens et un nombre n et decoupe cette liste en fragments de n tokens puis retourne une liste de listes de tokens"""
    i = 0 # i stocke l'indice auquel on est dans le rolling
    list_rolling = []
    while i-n < len(list_tokens):
        list_rolling.append(list_tokens[i:i+n])
        i+=n
    return list_rolling 

In [12]:
def rolling_operationnalisation(rolling_list, window, doc_name, nombre_bigrammes, feature):
    """fonction qui prend en parametre une liste de liste de token  et une fenetre et calcul le ratio type-token pour chaque liste et retourne une liste de liste de valeurs"""
    i = 1
    list_type_token = []
    list_entropy = []
    for list_tokens in rolling_list:
        if type(list_tokens) == list:
            table_freq = Counter(list_tokens)
            list_type_token.append(type_token(table_freq, window))
            list_entropy.append(entropy(table_freq, window, nombre_bigrammes))
            #On stocke chaque table de frequences pour les chunks de 1000 mots dans des dataframmes
            zipped = list(zip(table_freq.keys(), table_freq.values()))
            df = pd.DataFrame(zipped, columns=[feature, 'Nombre d\'apparitions'])
            df['Frequence d\'apparition'] = round(df['Nombre d\'apparitions']/sum(table_freq.values()),5)
            df.to_csv(r'tables_frequences/'+feature+'/'+doc_name+'_'+feature+str(i)+'freq.csv', index = False)
            i+=1
        
    return list_type_token, list_entropy

In [13]:
"""fonction qui prend en parametre une table de frequence et retourne le calcul du ratio type-token"""
def type_token(table_freq, window):
    #test de verification sur les valeurs recuperees
    if sum(table_freq.values()) == window:
        #On calcule le type_token ratio et on le retourne
        return round(len(table_freq)/sum(table_freq.values()),5)

In [14]:
def entropy(table_freq, window, nombre_bigrammes):
    """fonction qui prend en parametre une table de frequence et un nombre de bigramme et retourne leur entropie"""
    # initialisation de l'indice de shannon
    shannon_sum = 0
    #test de verification sur les valeurs recuperees
    if sum(table_freq.values()) == window:
        #On calcule l'entropie et on la retourne
        table_freq_select = dict(table_freq.most_common(nombre_bigrammes))
        for bigram in table_freq_select.keys():
            prop = table_freq_select[bigram]/window
            shannon_sum += prop * (math.log(prop, 5))   
        return round(shannon_sum * -1,2)

In [15]:
def get_n_most_common_features(list_tokens, n):
    table_freq = Counter(list_tokens).most_common(n)
    print(table_freq.keys())
    return table_freq.keys()

In [30]:
list_lemma_select

['le',
 'de',
 'un',
 'et',
 'il',
 'avoir',
 'à',
 'lui',
 'être',
 'son',
 'que',
 'l’',
 'ce',
 'je',
 'en',
 'd’',
 'qui',
 'pas',
 'ne',
 'vous',
 'se',
 'dans',
 'qu’',
 'tout',
 'pour',
 'faire',
 'dire',
 's’',
 'mon',
 'au',
 'éter',
 'n’',
 'sur',
 'plus',
 '-',
 'mais',
 'on',
 '–',
 'avec',
 'me',
 'par',
 'comme',
 'c’',
 'nous',
 'pouvoir',
 'si',
 'bien',
 'j’',
 'y',
 'voir',
 'aller',
 'même',
 'moi',
 'tu',
 'leur',
 'sans',
 'être',
 'm’',
 'vouloir',
 'savoir',
 'où',
 'venir',
 'homme',
 'autre',
 'ou',
 'à',
 'petit',
 'quelque',
 'prendre',
 'grand',
 'encore',
 'votre',
 'femme',
 'rien',
 'quand',
 'là',
 'main',
 'peu',
 'jour',
 'celui',
 'dont',
 'bon',
 'mettre',
 'aussi',
 'jeune',
 'heure',
 'cela',
 'devoir',
 'falloir',
 'été',
 'non',
 'croire',
 'temps',
 'oeil',
 'puis',
 'chose',
 'donc',
 'sous',
 'ça',
 'jamais',
 'Monsieur',
 'fois',
 'toujours',
 'passer',
 'notre',
 'après',
 'seul',
 'ton',
 'tête',
 'alors',
 'porte',
 'entendre',


In [31]:
list_bigram_lemma_select

['de_le',
 'dans_le',
 'd’_un',
 'qu’_il',
 'de_son',
 'à_le',
 'c’_être',
 'et_le',
 'sur_le',
 'que_le',
 'de_l’',
 'n’_avoir',
 'il_avoir',
 'et_de',
 'de_ce',
 'que_je',
 'j’_avoir',
 'tout_le',
 'je_ne',
 'y_avoir',
 'à_l’',
 'qu’_lui',
 'lui_avoir',
 'avoir_pas',
 'à_son',
 'par_le',
 'dans_un',
 'il_être',
 'il_ne',
 'avec_un',
 'avoir_un',
 'il_y',
 'il_se',
 'pour_le',
 'dans_son',
 'il_n’',
 'être_pas',
 'l’_avoir',
 'que_vous',
 'n’_être',
 'comme_un',
 'être_un',
 'je_être',
 'le_plus',
 'ne_pouvoir',
 'je_vous',
 'à_ce',
 'ce_être',
 'qu’_on',
 'et_il',
 'à_un',
 'vous_avoir',
 'de_mon',
 'c’_éter',
 'avoir_été',
 'un_peu',
 'il_falloir',
 'ce_que',
 'qui_avoir',
 'le_main',
 'de_tout',
 'je_me',
 'être_-ce',
 'avoir_le',
 'être_le',
 'et_je',
 'dire_il',
 'il_s’',
 'm’_avoir',
 'pas_de',
 'par_un',
 '-_être',
 'je_n’',
 'pouvoir_-',
 'qu’_un',
 'faire_un',
 'pas_le',
 'avoir_faire',
 'le_porte',
 'que_ce',
 'sur_son',
 'ce_qui',
 'avec_le',
 'dans_l’',
 'il_éter

In [32]:
list_trigram_lemma_select

['il_y_avoir',
 'n’_avoir_pas',
 'pouvoir_-_être',
 'n’_être_pas',
 'qu’_il_avoir',
 'je_n’_avoir',
 'lui_-_même',
 'c’_être_un',
 'que_j’_avoir',
 'il_n’_y',
 'n’_y_avoir',
 'ne_être_pas',
 'ce_qu’_il',
 'y_avoir_un',
 'n’_éter_pas',
 'il_n’_avoir',
 'ce_n’_être',
 'être_-ce_que',
 'je_ne_savoir',
 'c’_être_le',
 'ne_savoir_pas',
 'qu’_lui_avoir',
 'tout_à_coup',
 'qu’_il_ne',
 'au_milieu_de',
 'que_je_ne',
 'ne_vouloir_pas',
 'se_mettre_à',
 'qu’_il_n’',
 'de_tout_le',
 'ne_pouvoir_pas',
 'qu’_être_-ce',
 'ce_que_je',
 'qu’_il_être',
 'tout_le_monde',
 'c’_éter_un',
 'le_porte_de',
 'd’_un_voix',
 'n’_avoir_jamais',
 'je_ne_pouvoir',
 'le_jeune_fille',
 'être_-ce_pas',
 'lui_n’_avoir',
 'que_c’_être',
 'qu’_il_y',
 'je_l’_avoir',
 'n’_être_-ce',
 'de_ne_pas',
 'et_de_le',
 'que_je_être',
 'et_qu’_il',
 'un_de_ce',
 'à_l’_heure',
 'je_ne_être',
 'y_avoir_pas',
 'n’_avoir_plus',
 'ce_être_un',
 'je_me_être',
 'que_vous_avoir',
 'au_fond_de',
 'au_bout_de',
 'c’_éter_le',
 'avoi

In [33]:
list_pos_select

['NOUN',
 'VERB',
 'PRON',
 'ADP',
 'DET',
 'ADV',
 'ADJ',
 'PROPN',
 'CCONJ',
 'AUX',
 'SCONJ',
 'INTJ']

In [34]:
list_bigram_pos_select

['DET_NOUN',
 'PRON_VERB',
 'NOUN_ADP',
 'ADP_DET',
 'ADP_NOUN',
 'VERB_ADP',
 'NOUN_PRON',
 'VERB_DET',
 'PRON_PRON',
 'VERB_ADV',
 'NOUN_ADJ',
 'NOUN_VERB',
 'NOUN_DET',
 'NOUN_CCONJ',
 'VERB_PRON',
 'ADJ_NOUN',
 'PRON_AUX',
 'ADV_VERB',
 'NOUN_NOUN',
 'DET_ADJ',
 'ADP_PRON',
 'PRON_ADV',
 'SCONJ_PRON',
 'NOUN_ADV',
 'VERB_VERB',
 'AUX_VERB',
 'ADV_PRON',
 'ADP_VERB',
 'ADV_ADP',
 'ADJ_ADP',
 'CCONJ_PRON',
 'VERB_NOUN',
 'ADJ_PRON',
 'NOUN_PROPN',
 'ADV_DET',
 'ADP_PROPN',
 'ADV_ADV',
 'PROPN_PRON',
 'ADJ_DET',
 'ADV_ADJ',
 'VERB_SCONJ',
 'PROPN_ADP',
 'CCONJ_DET',
 'CCONJ_ADP',
 'PRON_ADP',
 'ADJ_CCONJ',
 'NOUN_SCONJ',
 'VERB_ADJ',
 'SCONJ_DET',
 'DET_PROPN',
 'ADP_ADJ',
 'VERB_PROPN',
 'PROPN_VERB',
 'AUX_ADV',
 'PRON_DET',
 'ADJ_VERB',
 'NOUN_AUX',
 'VERB_CCONJ',
 'ADP_ADV',
 'PROPN_PROPN',
 'PROPN_DET',
 'ADV_NOUN',
 'CCONJ_VERB',
 'ADV_SCONJ',
 'PROPN_NOUN',
 'PRON_NOUN',
 'PROPN_CCONJ',
 'ADV_AUX',
 'PROPN_ADV',
 'PRON_PROPN',
 'ADJ_ADV',
 'ADJ_ADJ',
 'CCONJ_ADV',
 'AUX_ADJ',
 

In [35]:
list_trigram_pos_select

['ADP_DET_NOUN',
 'DET_NOUN_ADP',
 'VERB_DET_NOUN',
 'NOUN_ADP_NOUN',
 'NOUN_ADP_DET',
 'PRON_PRON_VERB',
 'DET_NOUN_PRON',
 'PRON_VERB_ADP',
 'VERB_ADP_DET',
 'NOUN_PRON_VERB',
 'NOUN_DET_NOUN',
 'PRON_VERB_DET',
 'ADP_NOUN_ADP',
 'DET_NOUN_ADJ',
 'DET_NOUN_VERB',
 'DET_ADJ_NOUN',
 'PRON_VERB_ADV',
 'NOUN_PRON_PRON',
 'DET_NOUN_CCONJ',
 'DET_NOUN_DET',
 'VERB_ADP_NOUN',
 'PRON_AUX_VERB',
 'PRON_VERB_PRON',
 'ADP_NOUN_PRON',
 'DET_NOUN_NOUN',
 'VERB_PRON_VERB',
 'ADV_DET_NOUN',
 'PRON_VERB_VERB',
 'DET_NOUN_ADV',
 'ADV_PRON_VERB',
 'ADV_VERB_ADV',
 'PRON_ADV_VERB',
 'ADJ_DET_NOUN',
 'ADP_PRON_VERB',
 'NOUN_VERB_ADP',
 'CCONJ_DET_NOUN',
 'NOUN_ADJ_ADP',
 'ADP_NOUN_DET',
 'ADJ_NOUN_ADP',
 'PRON_PRON_PRON',
 'ADP_NOUN_CCONJ',
 'VERB_ADV_ADP',
 'PRON_PRON_AUX',
 'NOUN_VERB_DET',
 'ADP_DET_ADJ',
 'SCONJ_PRON_VERB',
 'SCONJ_DET_NOUN',
 'ADJ_ADP_DET',
 'NOUN_ADP_PRON',
 'NOUN_PRON_AUX',
 'ADP_VERB_DET',
 'NOUN_CCONJ_PRON',
 'VERB_ADP_PRON',
 'NOUN_ADP_VERB',
 'NOUN_ADP_PROPN',
 'VERB_PRON_PRO

In [46]:
def dict_freq_token(list_select, list_lemma):
    
    dict_result = dict.fromkeys(list_select)
    
    dict_temp = Counter(list_lemma)
        
    for key in dict_temp.keys():
        if key in dict_result.keys():
            dict_result[key] = dict_temp[key]/len(list_lemma)
    
    return dict_result

In [53]:
def moulinette(path_name, list_lemma_select, list_bigram_lemma_select, list_trigram_lemma_select, list_pos_select, list_bigram_pos_select, list_trigram_pos_select):
    """fonction main qui utilise les fonctions précédentes et tourne sur le corpus"""
    
    nombre_total_tokens = 0
 
    dict_results = {}

    df_main = pd.DataFrame()
    
    print("\n\nBEGIN PROCESSING CORPUS-----------")
    
    for doc in glob(path_name):
        
        print("\n\nBEGIN PROCESSING NOVEL-----------")

        doc_name = path.splitext(path.basename(doc))[0]
        date = doc_name.split("_")[0]
        print(doc_name)
        
        #On recupere le texte des romans sous forme de listes de lemmes et de pos grâce à spacy
        
        list_lemma_temp, list_pos_temp, nombre_tokens = pipeline_spacy(doc)
        
        print("PIPELINE SPACY ----------- OK")
        
        print("NOMBRE TOKENS = ", nombre_tokens)
        
        nombre_total_tokens += nombre_tokens
        
        list_bigram_lemma_temp = bigrammize(list_lemma_temp)
        list_trigram_lemma_temp = trigrammize(list_lemma_temp)
        
        list_bigram_pos_temp = bigrammize(list_pos_temp)
        list_trigram_pos_temp = trigrammize(list_pos_temp)
        
        dict_results.update(dict_freq_token(list_lemma_select, list_lemma_temp))
        dict_results.update(dict_freq_token(list_bigram_lemma_select, list_bigram_lemma_temp))
        dict_results.update(dict_freq_token(list_trigram_lemma_select, list_trigram_lemma_temp))
        dict_results.update(dict_freq_token(list_pos_select, list_pos_temp))
        dict_results.update(dict_freq_token(list_bigram_pos_select, list_bigram_pos_temp))
        dict_results.update(dict_freq_token(list_trigram_pos_select, list_trigram_pos_temp))
        
        dict_results["index"] = doc_name
        
        df_temp = pd.DataFrame(dict_results, index=[0])
        
        df_main = df_main.append(df_temp, ignore_index = True)
        
        print("PROCESS RESULTS ----------- OK")

        print("END PROCESSING NOVEL --------------\n\n")
        print("PROGRESSION ", round(nombre_total_tokens/238933674,2),'% COMPLETED\n')
        
    df_main.set_index("index", inplace = True)
    
    print("\n NOMBRE TOTAL TOKENS = ", nombre_total_tokens)   
    print("\n\n END PROCESSING CORPUS --------------\n\n")
    
    
    return df_main
    

In [50]:
starttime = timeit.default_timer()
df_main = moulinette(path_name, list_lemma_select, list_bigram_lemma_select, list_trigram_lemma_select, list_pos_select, list_bigram_pos_select, list_trigram_pos_select)
print("Le temps total d'execution en secondes est de : ", timeit.default_timer() - starttime)



BEGIN PROCESSING CORPUS-----------


BEGIN PROCESSING NOVEL-----------
2003_Echenoz-Jean_Au-Piano
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  48753
END PROCESSING NOVEL --------------


PROGRESSION  0.02 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1863_Feval-Paul_La-Fille-du-Juif-Errant
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  62340
END PROCESSING NOVEL --------------


PROGRESSION  0.05 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1893_Segur-Anatole-de_Les-Enfants-de-Paris
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  65702
END PROCESSING NOVEL --------------


PROGRESSION  0.07 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1839_Stendhal_Le-coffre-et-le-revenant
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  9787
END PROCESSING NOVEL --------------


PROGRESSION  0.08 % COMPLETED



BEGIN PROCESSING NOVEL-----------
2012_Winckler-Martin_En-souvenir-d-Andre
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  32711
END PROCESSING NOVEL --------------


PROGRESSION  0.

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  30775
END PROCESSING NOVEL --------------


PROGRESSION  1.23 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1868_Zaccone-Pierre_Le-Courrier-de-Lyon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  89589
END PROCESSING NOVEL --------------


PROGRESSION  1.26 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1845_Balzac-Honore-de_Gambara
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  26112
END PROCESSING NOVEL --------------


PROGRESSION  1.27 % COMPLETED



BEGIN PROCESSING NOVEL-----------
2009_Simon-Claude_Archipel-et-Nord
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  4932
END PROCESSING NOVEL --------------


PROGRESSION  1.28 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1950_Fourre-Maurice_La-Nuit-du-Rose-Hotel
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  77211
END PROCESSING NOVEL --------------


PROGRESSION  1.31 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1859_Dumas-Alexandre_Le-fils-du-forçat
PIPELINE SPACY ----------

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  90609
END PROCESSING NOVEL --------------


PROGRESSION  2.67 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1970_Deon-Michel_Les-poneys-sauvages
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  200658
END PROCESSING NOVEL --------------


PROGRESSION  2.76 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1943_Simenon-Georges_Les-dossiers-de-l-Agence-O
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  189442
END PROCESSING NOVEL --------------


PROGRESSION  2.84 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1958_San-Antonio_La-verite-en-salade
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  41880
END PROCESSING NOVEL --------------


PROGRESSION  2.85 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1948_Bealu-Marcel_L-Araignee-d-eau
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  36956
END PROCESSING NOVEL --------------


PROGRESSION  2.87 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1882_Albalat-Antoine_L-inassouvie-roman-intime
PIPE

END PROCESSING NOVEL --------------


PROGRESSION  4.09 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1874_Feval-Paul_La-Bande-Cadet-Les-Habits-Noirs_Tome-VIII
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  147439
END PROCESSING NOVEL --------------


PROGRESSION  4.15 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1948_Barjavel-Rene_Le-diable-l-emporte
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  86215
END PROCESSING NOVEL --------------


PROGRESSION  4.19 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1987_Green-Julien_Les-Pays-lointains
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  398797
END PROCESSING NOVEL --------------


PROGRESSION  4.35 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1898_Lesueur-Daniel_Levres-closes
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  39832
END PROCESSING NOVEL --------------


PROGRESSION  4.37 % COMPLETED



BEGIN PROCESSING NOVEL-----------
2001_Echenoz-Jean_Jerome-Lindon
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  9927
END PROCE

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  149168
END PROCESSING NOVEL --------------


PROGRESSION  5.51 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1926_Bourget-Paul_Le-danseur-mondain
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  56461
END PROCESSING NOVEL --------------


PROGRESSION  5.54 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1952_Calet-Henri_Un-Grand-Voyage
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  58965
END PROCESSING NOVEL --------------


PROGRESSION  5.56 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1894_Cim-Albert_Histoire-d-un-baiser
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  43272
END PROCESSING NOVEL --------------


PROGRESSION  5.58 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1883_Merouvel-Charles_Angele-Meraud
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  118311
END PROCESSING NOVEL --------------


PROGRESSION  5.63 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1866_Stapleaux-Leopold_Le-Chateau-de-la-rage.
PIPELINE SPACY ----

...

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  76464
END PROCESSING NOVEL --------------


PROGRESSION  92.71 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1928_Delly_Cœurs-ennemis-Laquelle
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  61953
END PROCESSING NOVEL --------------


PROGRESSION  92.73 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1890_Bazin-Rene_Les-Noellet
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  85994
END PROCESSING NOVEL --------------


PROGRESSION  92.77 % COMPLETED



BEGIN PROCESSING NOVEL-----------
2012_Damasio-Alain_Aucun-souvenir-assez-solide
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  81083
END PROCESSING NOVEL --------------


PROGRESSION  92.8 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1949_Delly_La-villa-des-serpents
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  41209
END PROCESSING NOVEL --------------


PROGRESSION  92.82 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1920_Boylesve-Rene_Le-carrosse-aux-deux-lezards-verts
PIPELINE S

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  167513
END PROCESSING NOVEL --------------


PROGRESSION  94.05 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1959_Delly_L'illusion-orgueilleuse-suivi-de-La-voie-divine
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  50079
END PROCESSING NOVEL --------------


PROGRESSION  94.07 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1963_San-Antonio_Le-gala-des-emplumes
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  51088
END PROCESSING NOVEL --------------


PROGRESSION  94.09 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1951_Delly_La-maison-des-belles-colonnes-L'accusatrice
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  50916
END PROCESSING NOVEL --------------


PROGRESSION  94.11 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1863_Pontmartin-Armand-de_Les-bruleurs-de-temples
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  136456
END PROCESSING NOVEL --------------


PROGRESSION  94.17 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1

END PROCESSING NOVEL --------------


PROGRESSION  95.34 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1905_Bazin-Rene_L-isolee
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  85848
END PROCESSING NOVEL --------------


PROGRESSION  95.38 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1846_Sand-George_La-Mare-au-Diable
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  46150
END PROCESSING NOVEL --------------


PROGRESSION  95.4 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1867_Vigny-Alfred-de_La-vie-et-la-mort-du-capitaine-Renaud-ou-La-canne-de-jonc--souvenirs-de-grandeur-militaire
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  32498
END PROCESSING NOVEL --------------


PROGRESSION  95.41 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1953_Simenon-Georges_Maigret-a-peur
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  53594
END PROCESSING NOVEL --------------


PROGRESSION  95.43 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1911_Apollinaire-Guillaume_Les-exploits-d-un-jeune-

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  29500
END PROCESSING NOVEL --------------


PROGRESSION  96.69 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1958_Kessel-Joseph_Le-lion
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  74562
END PROCESSING NOVEL --------------


PROGRESSION  96.72 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1918_Delly_Sous-le-masque
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  88063
END PROCESSING NOVEL --------------


PROGRESSION  96.76 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1978_Boileau-Narcejac_Carte-vermeil
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  62999
END PROCESSING NOVEL --------------


PROGRESSION  96.78 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1968_Guy-Des-Cars_La-revoltee
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  142092
END PROCESSING NOVEL --------------


PROGRESSION  96.84 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1951_Delly_Sous-l'œil-des-brahmes
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  1049

PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  49209
END PROCESSING NOVEL --------------


PROGRESSION  98.01 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1872_Gaboriau-Emile_La-Degringolade_Tome-3
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  105539
END PROCESSING NOVEL --------------


PROGRESSION  98.05 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1864_Gaboriau-Emile_L-argent-des-autres_I
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  114637
END PROCESSING NOVEL --------------


PROGRESSION  98.1 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1879_Bouvier-Alexis_La-femme-du-mort_Tome-II
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  59901
END PROCESSING NOVEL --------------


PROGRESSION  98.13 % COMPLETED



BEGIN PROCESSING NOVEL-----------
1824_Mazier-du-Heaume-Hippolyte_Voyage-d-un-jeune-grec-a-Paris_Volume-1
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  44080
END PROCESSING NOVEL --------------


PROGRESSION  98.14 % COMPLETED



BEGIN PROCESSING NOVEL-----------
18

In [51]:
df_main

,le,de,un,et,il,avoir,à,lui,être,son,...,CCONJ_SCONJ_NOUN,PROPN_VERB_AUX,VERB_CCONJ_CCONJ,ADV_NOUN_AUX,SCONJ_NOUN_AUX,PRON_ADJ_AUX,SCONJ_PROPN_DET,ADV_DET_DET,ADV_CCONJ_PROPN,VERB_VERB_AUX
index,,,,,,,,,,,,,,,,,,,,,
2003_Echenoz-Jean_Au-Piano,0.055371,0.066754,0.032462,0.016659,0.019914,0.013094,0.018679,0.009601,0.012571,0.012025,...,0.000095,None,0.000071,0.000024,None,0.000024,0.000071,0.000048,0.000143,None
1863_Feval-Paul_La-Fille-du-Juif-Errant,0.071686,0.056006,0.025387,0.021924,0.018092,0.019726,0.015096,0.007762,0.0128,0.012411,...,None,0.000058,0.000019,0.000019,0.000039,0.000078,0.000039,0.000039,0.000078,0.000078
1893_Segur-Anatole-de_Les-Enfants-de-Paris,0.070270,0.074120,0.024885,0.035188,0.020143,0.010624,0.018076,0.008057,0.013922,0.018379,...,0.000036,0.000036,0.000053,0.000018,None,0.000018,None,0.000018,0.000036,0.000018
1839_Stendhal_Le-coffre-et-le-revenant,0.058104,0.057741,0.020821,0.017552,0.024936,0.018642,0.018279,0.018884,0.013558,0.017916,...,None,None,None,None,0.000121,None,None,None,None,None
2012_Winckler-Martin_En-souvenir-d-Andre,0.050104,0.040242,0.026747,0.017232,0.020346,0.047474,0.018547,0.014879,0.015052,0.010588,...,None,0.000104,0.000069,0.000104,None,0.000035,0.000035,None,0.000069,0.000138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941_Ayme-Marcel_Le-travelingue,0.054117,0.051172,0.034685,0.024416,0.02059,0.019617,0.021799,0.013306,0.014962,0.016054,...,None,0.000066,0.000053,0.000053,0.000079,0.000026,0.000026,0.000066,0.000053,0.000053
1887_Malot-Hector_Ghislaine,0.064546,0.054961,0.025663,0.021358,0.028586,0.023401,0.020164,0.025799,0.014434,0.014162,...,0.000147,0.00001,0.00001,0.000073,0.000084,0.000031,0.000031,0.000031,0.000042,0.000042
1958_Beauvoir-Simone-de_Memoires-d-une-jeune-fille-rangee,0.051237,0.048684,0.031195,0.019573,0.018469,0.016743,0.021436,0.015501,0.005853,0.011588,...,0.000021,0.000028,0.000035,0.000014,0.000041,0.000014,0.000048,0.000055,0.000007,0.000014


In [52]:
df_main.to_csv(r'features.csv', index = True)

MAX_PROCESS

1843_Sand-George_La-Comtesse-de-Rudolstadt
244685 token
1,3 MB (1 276 797 octets)

PROB

1952_Rebatet-Lucien_Les-Deux-Etendards
602336
3,0 MB (3 034 787 octets)

REQUIRED SIZE = 2 920 947

ACTUAL SIZE + 2 000 000

MAX_OVERALL
4,1 MB (4 086 852 octets)
846835

In [46]:
nb_max = 0
for f in os.listdir('corpus_main_txt/'):
    if nb_max < os.path.getsize('corpus_main_txt/'+f):
        nb_max = os.path.getsize('corpus_main_txt/'+f)
        print(nb_max, f)

242432 2003_Echenoz-Jean_Au-Piano.txt
305990 1863_Feval-Paul_La-Fille-du-Juif-Errant.txt
331094 1893_Segur-Anatole-de_Les-Enfants-de-Paris.txt
473049 1882_Feval-Paul_Veillees-de-la-famille.txt
476092 1883_Greville-Henry_Angele.txt
536178 1836_Musset-Alfred-de_La-Confession-d-un-enfant-du-siecle.txt
678395 1926_Leroux-Gaston_Le-coup-d-Etat-de-Cheri-Bibi.txt
706198 1912_Zevaco-Michel_Le-Rival-du-Roi.txt
1203666 1892_Zola-Emile_La-debacle.txt
1276797 1843_Sand-George_La-Comtesse-de-Rudolstadt.txt
3034787 1952_Rebatet-Lucien_Les-Deux-Etendards.txt
4086852 1850_Dumas-Alexandre_Le-Vicomte-de-Bragelonne.txt
